# Agentic AI MCP - Server Only

This notebook starts an MCP server that exposes tools.
Run this on the machine where you want to host the tools.

Another machine can connect to this server using the client notebook.

## Step 1: Setup

In [1]:
# install if needed
# !uv pip install agentic_ai_mcp==0.5.3

import agentic_ai_mcp

print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.5.3


## Step 2: Define Your Tools

In [2]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


def greet(name: str, times: int = 1) -> str:
    """Greet someone."""
    return ("Hello, " + name + "! ") * times


def grid_scan(
    x_start: float,
    x_end: float,
    y_start: float,
    y_end: float,
    step_size: float,
    scan_speed: float = 1.0,
) -> dict:
    """Perform a grid scan over a defined XY area on the AFM sample surface."""
    return {"status": "ok", "scan_type": "grid", "points_scanned": 100}


def line_scan(
    x_start: float,
    x_end: float,
    y_position: float,
    num_points: int = 256,
    scan_speed: float = 1.0,
) -> dict:
    """Perform a single line scan along the X axis at a fixed Y position."""
    return {"status": "ok", "scan_type": "line", "points_scanned": num_points}


def raster_scan(
    width: float,
    height: float,
    resolution: int = 512,
    scan_angle: float = 0.0,
    scan_speed: float = 1.0,
) -> dict:
    """Perform a raster scan over a rectangular area with defined resolution."""
    return {"status": "ok", "scan_type": "raster", "resolution": resolution}


def set_tip_position(
    x: float,
    y: float,
    z: float = 0.0,
) -> dict:
    """Move the AFM tip to a specific XYZ position on the sample."""
    return {"status": "ok", "tip_position": {"x": x, "y": y, "z": z}}


def set_scan_parameters(
    setpoint: float,
    p_gain: float = 1.0,
    i_gain: float = 0.5,
    scan_rate: float = 1.0,
) -> dict:
    """Set AFM feedback controller parameters including setpoint and PID gains."""
    return {"status": "ok", "setpoint": setpoint, "p_gain": p_gain, "i_gain": i_gain}


def get_force_curve(
    x: float,
    y: float,
    z_start: float = 100.0,
    z_end: float = -10.0,
    num_points: int = 512,
) -> dict:
    """Acquire a force-distance curve at a specified XY location on the sample."""
    return {"status": "ok", "location": {"x": x, "y": y}, "num_points": num_points}


def approach_surface(
    target_setpoint: float,
    max_z_travel: float = 10.0,
) -> dict:
    """Automatically approach the AFM tip toward the sample surface until setpoint is reached."""
    return {"status": "ok", "approached": True, "z_position": 2.5}


def retract_tip(
    retract_distance: float = 50.0,
) -> dict:
    """Retract the AFM tip away from the sample surface by a specified distance."""
    return {"status": "ok", "retracted": True, "retract_distance": retract_distance}


def get_scan_image(
    channel: str = "height",
) -> dict:
    """Retrieve the most recently acquired scan image for a specified data channel."""
    return {"status": "ok", "channel": channel, "image_shape": [512, 512]}


def calibrate_scanner(
    calibration_mode: str = "xy",
) -> dict:
    """Run scanner calibration routine for XY or Z axis using reference sample."""
    return {"status": "ok", "calibration_mode": calibration_mode, "calibrated": True}

## Step 3: Create AgenticAI and Register Tools

Configure the host/port. 

In [3]:
from agentic_ai_mcp import AgenticAI

ai = AgenticAI(host="0.0.0.0", port=8888)

# register your tools
ai.register_tool(add)
ai.register_tool(multiply)
ai.register_tool(greet)
ai.register_tool(grid_scan)
ai.register_tool(line_scan)
ai.register_tool(raster_scan)
ai.register_tool(set_tip_position)
ai.register_tool(set_scan_parameters)
ai.register_tool(get_force_curve)
ai.register_tool(approach_surface)
ai.register_tool(retract_tip)
ai.register_tool(get_scan_image)
ai.register_tool(calibrate_scanner)

print(f"Registered tools: {ai.tools}")

Registered tools: ['add', 'multiply', 'greet', 'grid_scan', 'line_scan', 'raster_scan', 'set_tip_position', 'set_scan_parameters', 'get_force_curve', 'approach_surface', 'retract_tip', 'get_scan_image', 'calibrate_scanner']


## Step 4: Start the MCP Server

This will start the server in background. The server URL will be:
- Local: `http://127.0.0.1:8888/mcp`
- Remote: `http://<your-ip>:8888/mcp`

Give this URL to the client notebook.

In [4]:
# start server in background
ai.run_mcp_server()

# get your IP for remote connections
import socket

hostname = socket.gethostname()
try:
    local_ip = socket.gethostbyname(hostname)
except:
    local_ip = "<your-ip>"

print("\nServer is ready!")
print("Local URL:  http://127.0.0.1:8888/mcp")
print(f"Remote URL: http://{local_ip}:8888/mcp")

INFO:     Started server process [127816]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8888): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
/home/cloud/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/popen_fork.py:67: DeprecationWarning: This process (pid=127800) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()



Server is ready!
Local URL:  http://127.0.0.1:8888/mcp
Remote URL: http://127.0.1.1:8888/mcp


## Step 5: Stop the Server

Run this cell to stop the server when you're done.

In [5]:
ai.stop_mcp_server()